In [ ]:
#Importing Libraries
import cv2
import random
import numpy as np
import pandas as pd
from deepface import DeepFace #Lirary for analysing emotion

In [14]:
#Loading Music Dataset From Folder
data = pd.read_csv('C:/Users/umendra/OneDrive/Documents/DataSet/genres_v2.csv')
data = data.dropna()
data = data.loc[:,["acousticness","valence","song_name"]]
data.head()

,acousticness,valence,song_name
0,0.0598,0.3890,Mercury: Retrograde
1,0.4010,0.1240,Pathology
2,0.0138,0.0391,Symbiote
3,0.0237,0.1750,ProductOfDrugs (Prod. The Virus and Antidote)
4,0.2170,0.5910,Venom


In [3]:
#Computation over music datset
mean_ac = np.mean(data.loc[:,"acousticness"])
mean_va = np.mean(data.loc[:,"valence"])
print(mean_ac,mean_va)

0.16728227078442431 0.41738458571494885


In [7]:
#Performing music recommendation over any given image
im = cv2.imread('./faceEmoji/face0.jpg') #Loading Image
pre = DeepFace.analyze(im,actions=['emotion']) #Analysing Emotions
print("Your dominant emotion is:",pre['dominant_emotion'])
predict_music(pre)

Your dominant emotion is: neutral
Enter the number of songs you want:
10
Your songs are as follows:

['Come Up', 'Pumpkins Scream In the Dead of Night (feat. shinigami)', 'Rider Pt. 2', 'No Sleep', "Runnin'", 'Ricky Bobby Six Speed', 'Got My Mind Made Up', 'Music Is My Wife', 'Angelic Hoodrat']


In [6]:
#Function for music recommendation

def predict_music(pre):
    em = {'angry':pre['emotion']['angry'],'happy':pre['emotion']['happy'],'sad':pre['emotion']['sad'],'neutral':pre['emotion']['neutral']}
    em = dict(sorted(em.items(),key=lambda x:x[1],reverse=True))
    l_keys = list(em.keys())
    if(l_keys[0]=="anger"):
        aux_dta = data[(data['acousticness']>mean_ac) & (data['valence']<mean_va)]
        aux_dta = aux_dta.sort_values(by='valence')
    elif(l_keys[0]=="happy"):
        aux_dta = data[(data['acousticness']>mean_ac) & (data['valence']>mean_va)]
        aux_dta = aux_dta.sort_values(by='acousticness',ascending=False)
    elif(l_keys[0]=="sad"):
        aux_dta = data[(data['acousticness']<mean_ac) & (data['valence']<mean_va)]
        aux_dta = aux_dta.sort_values(by='acousticness')
    else:    
        aux_dta = data[(data['acousticness']<mean_ac) & (data['valence']>mean_va)]
    
    print("Enter the number of songs you want:")
    count = int(input())
    l = []
    for i in range(1,count):
        l.append(random.randint(1,len(aux_dta.index)))   
    pc=pd.DataFrame(aux_dta.iloc[l])
    p = list(pc['song_name'])
    print("Your songs are as follows:\n")
    print(p)

In [8]:
#Loading file for face detection
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades+"haarcascade_frontalface_default.xml")

In [13]:
#Performing real time emotion-detection and music recommendation
#For music recommendation enter the key:q

cap = cv2.VideoCapture(0)
while True:
    ret,img = cap.read();
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,1.1,4)
    pre = DeepFace.analyze(img,actions=['emotion'])
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    cv2.putText(img,pre['dominant_emotion'],(50,50),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),2,cv2.LINE_AA)
    cv2.imshow("Frame",img)
    if cv2.waitKey(25) & 0xFF==ord('q'):
        cv2.imwrite("./faceEmoji/face-emotion.jpg",img)
        print("Your dominant emotion is:",pre['dominant_emotion'])
        predict_music(pre)
        break
cv2.destroyAllWindows()        

Your dominant emotion is: neutral
Enter the number of songs you want:
10
Your songs are as follows:

["I Can't", 'SPACE PUNKS', 'Planting The Roots Only To Fall Out The Tree', 'Represent', 'Killing Me Softly With His Song', 'LIGHTSKIN SH*T (feat. Future & jetsonmade)', 'Spotlight', 'I Really Like You', 'company']
